In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [8]:
from util.general import load_csv_pandas
from util.general import load_csv
from util.executable import get_destination

In [9]:
from util.build_dataset import build_origin_price

origin_p_df, key = build_origin_price(date="201908",bucket_name="production-bobsim")

2020-04-14 20:16:07:manage:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'


**for target, fix 'retail price'** 

In [10]:
retail_df = origin_p_df[origin_p_df['class']=='소비자가격'].drop('class', axis=1)

In [11]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81737 entries, 0 to 97408
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                81737 non-null  datetime64[ns]
 1   region              81737 non-null  object        
 2   unit_name           81737 non-null  object        
 3   grade               81737 non-null  object        
 4   standard_item_name  81737 non-null  object        
 5   price               81737 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(4)
memory usage: 4.1+ MB


### null value count

In [12]:
retail_df.isnull().sum()

date                  0
region                0
unit_name             0
grade                 0
standard_item_name    0
price                 0
dtype: int64

In [14]:
retail_df.head()

,date,region,unit_name,grade,standard_item_name,price
0,2019-08-01,서울서부,20KG,상(1등급),벼,54300
1,2019-08-01,서울서부,20KG,상(1등급),벼,53200
2,2019-08-01,서울서부,20KG,상(1등급),벼,50600
5,2019-08-01,서울,20KG,상(1등급),벼,54000
6,2019-08-01,서울,20KG,상(1등급),벼,47900


In [ ]:
    def get_unit(unit_name):
        # TODO: handle no supported unit_name
        return {
            '20KG': 200, '1.2KG': 12, '8KG': 80, '1KG': 10, '1KG(단)': 10, '1KG(1단)': 10,
            '500G': 5, '200G': 2, '100G': 1, '30개' : 30, '600G' : 6, '5마리' : 5,
            '10마리': 10, '2마리': 2, '1마리': 1, '5KG': 50 , '2KG' : 20 ,
            '10개': 10, '1개': 1,
            '1L': 10,
            '1속': 1,
            '1포기': 1,
        }.get(unit_name, 1) 
    def by_unit( df: pd.DataFrame):
        """
            transform unit
        :return: transformed pd DataFrame
        """
        return df.assign(
            조사단위명=lambda r: r.조사단위명.map(
                lambda x: get_unit(x)
            )
        ).assign(
            당일조사가격=lambda x: x.당일조사가격 / x.조사단위명
        ).drop("조사단위명", axis=1)

In [ ]:
retail_converted_df = by_unit(filtered_df)
retail_converted_df.info()

# Distribution

## load cleaned data

In [1]:
from util.build_dataset import build_master, build_origin_price, build_process_price

clean_p, c_key = build_origin_price(bucket_name="production-bobsim", date="201908", prefix="clean")
process_p, p_key = build_process_price(bucket_name="production-bobsim", date="201908")

2020-04-14 20:00:33:manage:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'


AttributeError: 'DataFrame' object has no attribute 'to_list'

In [ ]:
from util.visualize import draw_hist
from util.s3_manager.manage import S3Manager
import numpy as np

draw_hist(np.log1p(clean_p["price"].rename("log transformed price")))
S3Manager("production-bobsim").save_plt_to_png(key="food_material_price_predict_model/image/log_price_hist.png")

In [ ]:
price = clean_p["price"]
stdized_price = process_p["price"].rename("stdized_price")
log_transformed_price = np.log1p(clean_p["price"].rename("log_transformed_price"))

In [ ]:
import pandas as pd
from scipy.stats import skew

price_df = pd.concat([price, stdized_price, log_transformed_price], axis=1)
print(price_df.columns)
pd.Series(skew(price_df), index=price_df.columns).rename("skew").to_frame().T

## categorical

In [ ]:
from util.geography import compose_geography
composed = compose_geography(clean_p["region"])
composed

In [15]:
(retail_df["region"].unique()).size

24

In [16]:
(retail_df["region"].value_counts()/retail_df["region"].value_counts().sum()*100).rename("%").to_frame().sort_values(by="%")

,%
강원,1.592914
청주,1.734832
경기,1.788664
전주,1.821696
세종,1.869410
경북안동,1.912231
경기의정부,1.940370
강원춘천,2.176493
충북청주,2.706241
전북전주,2.725816


In [17]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(440/retail_df["standard_item_name"].value_counts().sum())
df =(retail_df["standard_item_name"].value_counts()/retail_df["standard_item_name"].value_counts().sum()*100).rename("%").to_frame().sort_values(by="%")

0.005383119028102328


In [18]:
high_list=df.sort_values(by='%', ascending =False).head(49).index

In [19]:
high_list

Index(['고추', '돈육', '한우', '배추', '상추', '포도', '오이', '호박', '사과', '조미제품', '땅콩', '무', '감자', '고구마', '대파', '당근', '생강', '파프리카', '양배추', '바나나', '새송이', '참깨', '팽이버섯', '레몬', '열무', '양파', '팥', '피망(단고추)', '수박', '참외', '배', '콩', '메론', '깻잎', '토마토', '고등어류', '파인애플', '쇠고기', '건가공(해조)류', '천연조미료류', '찹쌀', '젓갈(액젓)류', '벼', '아몬드', '건제품', '우유', '호두', '조란', '쪽파'], dtype='object')

In [22]:
filter_df = retail_df[~retail_df["standard_item_name"].isin(high_list)]

In [32]:
filter_df

,date,region,unit_name,grade,standard_item_name,price
245,2019-08-01,서울서부,500G,상(1등급),녹두,6230
246,2019-08-01,서울서부,500G,중(2등급),녹두,5830
247,2019-08-01,서울서부,500G,상(1등급),녹두,7100
248,2019-08-01,서울서부,500G,중(2등급),녹두,6000
249,2019-08-01,서울서부,500G,상(1등급),녹두,7330
...,...,...,...,...,...,...
97404,2019-08-30,경북포항,5마리,중(2등급),새우류,5620
97405,2019-08-30,경북안동,5마리,중(2등급),새우류,4800
97406,2019-08-30,경남창원,5마리,중(2등급),새우류,4750
97407,2019-08-30,경남창원,5마리,중(2등급),새우류,4400


In [25]:
filter_series = retail_df[~retail_df["standard_item_name"].isin(high_list)]['standard_item_name']

In [26]:
drop_dup = filter_series.drop_duplicates()

In [33]:
drop_list = drop_dup.to_list()

In [34]:
drop_list

['녹두',
 '시금치',
 '미나리',
 '느타리버섯',
 '복숭아',
 '참다래(키위)',
 '오렌지',
 '방울토마토',
 '체리',
 '블루베리',
 '닭고기',
 '꽁치류',
 '갈치류',
 '명태류',
 '오징어류',
 '멸치류',
 '건가공(포)류',
 '김류',
 '조기류',
 '젓갈(일반)류',
 '전복류',
 '새우류']

In [30]:
replace_df = retail_df.replace(drop_list,'그외')

In [31]:
replace_df

,date,region,unit_name,grade,standard_item_name,price
0,2019-08-01,서울서부,20KG,상(1등급),벼,54300
1,2019-08-01,서울서부,20KG,상(1등급),벼,53200
2,2019-08-01,서울서부,20KG,상(1등급),벼,50600
5,2019-08-01,서울,20KG,상(1등급),벼,54000
6,2019-08-01,서울,20KG,상(1등급),벼,47900
...,...,...,...,...,...,...
97404,2019-08-30,경북포항,5마리,중(2등급),그외,5620
97405,2019-08-30,경북안동,5마리,중(2등급),그외,4800
97406,2019-08-30,경남창원,5마리,중(2등급),그외,4750
97407,2019-08-30,경남창원,5마리,중(2등급),그외,4400


In [38]:
a = list(retail_df["standard_item_name"].unique())

In [40]:
def filter_sparse(df: pd.DataFrame, column: str, sparse_list: list, to_word: str = "others"):
    filter_list = list(set(list(df[column].unique())) - set(sparse_list))
    return df.replace(filter_list, to_word)


In [ ]:
a = filter_sparse(retail_df , column="standard_item_name", sparse_list= high_list)

In [ ]:
sim = clean_p["standard_item_name"].value_counts()
sparse_index = sim[sim < 440].index

filtered = clean_p["standard_item_name"].apply(lambda x: "그외" if x in sparse_index else x)
filtered

In [ ]:
(filtered.value_counts()/filtered.value_counts().sum()*100).rename("%").to_frame().sort_values(by="%")

In [ ]:
from util.visualize import draw_hist

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='darkgrid')

sns.countplot(x = 'region',
              data = clean_p)
plt.show()

In [ ]:
draw_hist(composed.value_counts())

# Compare: numeric value( nomal vs log1p ) // Base skew_values

In [ ]:
orgin_df=pd.read_csv('https://production-bobsim.s3.ap-northeast-2.amazonaws.com/public_data/public_price/origin/csv/201908.csv', encoding='euc-kr')

In [ ]:
columns = [
            "조사일자", "조사구분명",
            "표준품목명", "조사가격품목명", "표준품종명", "조사가격품종명",
            "조사등급명", "조사단위명", "당일조사가격", "조사지역명"
        ]

In [ ]:
filtered_df=orgin_df[columns]

## Check numeric type

In [ ]:
filtered_df.info()

### hist using displot

In [ ]:
# see distribution of numecric type 
plt.title('price Histogram')
sns.distplot(data_df['당일조사가격'])


In [ ]:
# check skew 
from scipy.stats import skew
features_index = filtered_df.dtypes[filtered_df.dtypes != 'object'].index

# house_df에 컬럼 index를 [ ]로 입력하면 해당하는 컬럼 데이터 셋 반환. apply lambda로 skew( )호출 
skew_features = filtered_df[features_index].apply(lambda x : skew(x))

# skew 정도가 1 이상인 컬럼들만 추출. 
skew_features_top = skew_features[skew_features > 1]
print(skew_features_top.sort_values(ascending=False))

## 로그변환

In [ ]:
filtered_df[skew_features_top.index] = np.log1p(filtered_df[skew_features_top.index])

## 분포도 확인 ( 결정한 이유 )

In [ ]:
# see distribution of numecric type 
plt.title('price Histogram')
sns.distplot(filtered_df['당일조사가격'])

In [ ]:
skew(filtered_df['당일조사가격'])

### datetime 타입에서 년, 월, 일, 시간 추출


In [ ]:
filtered_df['조사일자'] = filtered_df.조사일자.apply(pd.to_datetime)

In [ ]:
filtered_df['year']= filtered_df.조사일자.apply(lambda x : x.year)
filtered_df['month']=filtered_df.조사일자.apply(lambda x : x.month)
filtered_df['day']=filtered_df.조사일자.apply(lambda x: x.day)

In [ ]:
filtered_df.info()

In [ ]:
temp1=pd.read_csv('https://production-bobsim.s3.ap-northeast-2.amazonaws.com/public_data/public_price/origin/csv/201908.csv', encoding='euc-kr')
columns = [
            "조사일자", "조사구분명",
            "표준품목명", "조사가격품목명", "표준품종명", "조사가격품종명",
            "조사등급명", "조사단위명", "당일조사가격", "조사지역명"
        ]

In [ ]:
def unit_fit(unit_name):
    unit=0
    if unit_name=='20KG' : unit= 200
    elif unit_name=='1.2KG' : unit= 12
    elif unit_name=='8KG' : unit= 80
    elif unit_name=='1KG' : unit=10
    elif unit_name=='1KG(단)' : unit=10
    elif unit_name=='1KG(1단)': unit=10
    elif unit_name=='10개': unit=10
    elif unit_name=='10마리': unit=10
    elif unit_name=='1L' : unit=10 
    elif unit_name=='100G': unit= 1
    elif unit_name=='1개' : unit= 1
    elif unit_name=='1마리' : unit= 1
    elif unit_name=='1속' : unit= 1
    elif unit_name=='1포기' : unit= 1
    elif unit_name=='200G' :unit=2
    elif unit_name=='2마리' : unit= 2
    elif unit_name=='500G' : unit= 5
    else: unit = 6
    
    return unit

In [ ]:
temp2=temp1[temp1['조사구분명']=='소비자가격']
temp2 =temp2[['조사일자', '조사가격품목명','표준품종명','조사가격품종명','표준품목명','조사지역명','조사단위명','당일조사가격']]


In [ ]:
temp2['price_unit']= temp2['조사단위명'].apply(lambda x : unit_fit(x))
temp2['averge_price']= temp2['당일조사가격']/temp2['price_unit']

In [ ]:
# see distribution of numecric type 
plt.title('price Histogram')
sns.distplot(temp2['averge_price'])

In [ ]:
skew(temp2['averge_price']) # 소비자가격 skew

In [ ]:
skew(temp1['당일조사가격']) # 모든가격 skew

In [ ]:
temp2['averge_price'] = np.log1p(temp2['averge_price'])
print('로그변화 후 skew 값')
skew(temp2['averge_price'])

In [ ]:
plt.title('price Histogram with log1p')
sns.distplot(temp2['averge_price'])